# Reto. Movilidad
## Intersección de cuatro vias

### Equipo 2:
- Carlos Tadeo Pérez Capistrán, A01197315
- David Fernando Armendáriz Torres, A01570813
- Nicolás Herrera Hernandez, A01114972




## Imports
 

In [1]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model, space 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import SingleGrid, MultiGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## Crear el modelo y los agentes


In [2]:
# Agente de la aspiradora o robot limpiador
class Automovil(Agent):
  def __init__(self, unique_id, model, direction, rotation):
    super().__init__(unique_id, model)
    # Direccion 0 = right, 1 = left, 2 = up, 3 = down
    self.direction = direction
    self.color = np.random.choice([5,4])
    self.rotation = rotation


  def step(self):
    up = (self.pos[0] + 1, self.pos[1])
    down = (self.pos[0] - 1, self.pos[1])
    left = (self.pos[0], self.pos[1] - 1)
    right = (self.pos[0], self.pos[1] + 1)
    dir = { 0: right, 1: left, 2: down, 3: up}
    next_move = (dir[self.direction])
    if self.model.grid.out_of_bounds(next_move):
      self.next_move = self.model.grid.torus_adj(next_move)
    else: 
      self.next_move = next_move

  def advance(self):
    up = (self.pos[0] - 1, self.pos[1])
    down = (self.pos[0] + 1, self.pos[1])
    left = (self.pos[0], self.pos[1] + 1)
    right = (self.pos[0], self.pos[1] - 1)
    dir = { 0: right, 1: left, 2: down, 3: up}
    last_move = (dir[self.direction])
    for (content, x, y) in self.model.grid.coord_iter():
      if (x,y) == (3, 3):
        tfLeft = content.color
      elif (x,y) == (3, 6):
        tfUp = content.color
      elif (x,y) == (6, 3):
        tfDown = content.color
      elif (x,y) == (6, 6):
        tfRight = content.color

    neighbours = self.model.grid.get_neighbors(self.pos, moore=False, include_center=False)

    cross = {(4, 4), (4, 5), (5, 5), (5, 4)}
    semaforo = {0: tfRight, 1: tfLeft, 2: tfUp, 3: tfDown}
    canMove = True

    for n in neighbours:
      if isinstance(n, Semaforo):
        if semaforo[self.direction] == 3:
          canMove = True
        else:
          canMove = False
    if self.pos in cross or last_move in cross:
      canMove = True
    if self.model.grid.is_cell_empty(self.next_move) and canMove:
      self.model.grid.move_agent(self, self.next_move)
      self.pos = self.next_move




# Agente del semaforo
class Semaforo(Agent):
  def __init__(self, unique_id, model, color, dir, rotation):
    super().__init__(unique_id, model)
    # verde = 3, amarillo = 2, rojo = 1
    self.color = color
    self.dir = dir
    self.rotation = rotation

# Agente del terreno
class Terreno(Agent):
  def __init__(self, unique_id, model, rotation):
    super().__init__(unique_id, model)
    self.color = 6
    self.rotation = rotation

# Modelo
class Modelo(Model):
  def __init__(self, width, height, n):
    self.grid = SingleGrid(width, height, True)
    self.schedule = SimultaneousActivation(self)
    self.running = True
    self.num_agents = n
    self.timer = 10
    self.counter = 0

    s1 = Semaforo(1, self, 3, 1, 0)
    self.schedule.add(s1)
    self.grid.place_agent(s1, (3, 3))
    s2 = Semaforo(2, self, 1, 2, 0)
    self.schedule.add(s2)
    self.grid.place_agent(s2, (3, 6))
    s3 = Semaforo(3, self, 1, 3, 0)
    self.schedule.add(s3)
    self.grid.place_agent(s3, (6, 3))
    s4 = Semaforo(4, self, 1, 0, 0)
    self.schedule.add(s4)
    self.grid.place_agent(s4, (6, 6))
    dir = 0
    rotation = 0
    out_posiciones = []
    self.traffic_lights = [s1, s2, s3, s4]
    for i in range(width-1):
        out_posiciones.append((i, 4))
        out_posiciones.append((i, 5))
        out_posiciones.append((5, i))
        out_posiciones.append((4, i))
    cross = {(4, 4), (4, 5), (5, 5), (5, 4)}
    out_posiciones = [pos for pos in out_posiciones if pos not in cross]
    out = self.random.sample(out_posiciones, k = NUMBER_CARS)
    agent_count = 0
    for pos in out:
      if pos[0] == 4:
        dir = 0
        rotation = 90
      elif pos[0] == 5:
        dir = 1
        rotation = -90
      elif pos[1] == 4:
        dir = 2
        rotation = 0
      elif pos[1] == 5:
        dir = 3
        rotation = 180

      c = Automovil(1000+agent_count, self, dir, rotation)
      self.grid.place_agent(c, pos)
      self.schedule.add(c)
      agent_count += 1
          
    for (content, x, y) in self.grid.coord_iter():
      if self.grid.is_cell_empty((x,y)) and x not in [4, 5] and y not in [4, 5]:
        a = Terreno((x, y), self, 0)
        self.grid.place_agent(a, (x, y))
        self.schedule.add(a)

    # Aquí definimos el colector de datos para obtener el grid completo y los parametros de los agentes.
    self.datacollector = DataCollector(
    model_reporters={"Grid": self.get_grid}, agent_reporters={"Color" : "color", "position" : "pos", "Rotation" : "rotation"}    )

  def step(self):
    self.datacollector.collect(self)
    self.schedule.step()
    self.directions = [0, 1, 2, 3]
    if self.counter < self.timer:
      self.counter += 1
    else:
      for i, direction in enumerate(self.directions):
        if self.traffic_lights[direction].color == 3:
          self.counter = 0
          self.traffic_lights[direction].color = 1

          next_i = i + 1 if i < len(self.directions) - 1 else 0
          self.traffic_lights[self.directions[next_i]].color = 3
          break

  def get_grid(self):

      # Generamos la grid para contener los valores
      grid = np.zeros((self.grid.width, self.grid.height))

      # Asignamos una celda a cada uno de los elementos de la grilla
      for cell in self.grid.coord_iter():
          cell_content, x, y = cell
          if self.grid.is_cell_empty((x, y)):
            grid[x][y] = 0
          else:
            grid[x][y] = cell_content.color

      return grid


# Ejecución del modelo
A continuación corremos el modelo con los valores proporcionados.

In [3]:
# Definimos el tamaño del Grid, procentaje de celdas sucias y num de agentes


# Definimos el tiempo limite de ejecucion
NUMBER_CARS = 8

# Creamos el modelo con las dimensione
empty_model = Modelo(10, 10, NUMBER_CARS)

# Registramos el tiempo de inicio

# Corremos el modelo mientras que no se cumpla el tiempo limite y que todas las celdas no esten limpias
for i in range(50):
  empty_model.step()

# Guardamos el tiempo que le tomó correr al modelo.



# Datos a analizar
Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información. Con esta información podremos calcular lo siguiente:

- Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).
- Porcentaje de celdas limpias después del termino de la simulación.
- Número de movimientos realizados por todos los agentes.

In [4]:
# Obtenemos la información recolectada del modelo, y creamos nuestro archivo JSON
import json
all_grid = empty_model.datacollector.get_model_vars_dataframe()
agent_vars = empty_model.datacollector.get_agent_vars_dataframe().reset_index()
agent_var = agent_vars[agent_vars['Color'] != 6].dropna()
agent_var[['posicionX', 'posicionY']] = agent_var['position'].astype(str).str.split(",", expand=True)
agent_var['posicionX'] = agent_var['posicionX'].str.replace("(", "")
agent_var['posicionY'] = agent_var['posicionY'].str.replace(")", "")
agent_var.drop('position', inplace=True, axis=1)
ids = agent_var.AgentID.unique()
agent_var = agent_var.astype("int32")
agents = agent_var[['AgentID','Rotation']].values.tolist()
agent_var.drop('Rotation', inplace=True, axis=1)
steps = agent_var.to_dict(orient='records')
jsonTest = {}
out_agents = []
out_ids = []

out_steps = []
for id, rot in agents:
    out_ids.append({"id" : id, "rotation": rot})
for i in range(NUMBER_CARS+4):
    out_agents.append(out_ids[i])
for step in steps:
    out_steps.append({"agentInfo" : step})
jsonTest["agents"] = out_agents
jsonTest["stepsQuantity"] = len(all_grid)
jsonTest["steps"] = out_steps
with open(r"D:\Todo\5to Semestre\TC2008B\Unity\Connection_Request\Python Project\simulation.json", "w") as outfile:
    json.dump(jsonTest, outfile)


C:\Users\nicol\AppData\Local\Temp\ipykernel_11480\2336057778.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  agent_var['posicionX'] = agent_var['posicionX'].str.replace("(", "")
C:\Users\nicol\AppData\Local\Temp\ipykernel_11480\2336057778.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  agent_var['posicionY'] = agent_var['posicionY'].str.replace(")", "")


# Graficación
Graficamos la información usando `matplotlib`

In [5]:
%%capture
# Obtenemos el numero de pasos del recolector de datos
steps = len(all_grid)
cmap = matplotlib.cm.get_cmap('viridis', 7) # Puede ser cualquier otra
cmap = cmap(np.linspace(0, 1, 7))
cmap[0] = np.array([50/256, 50/256, 50/256, 1])   # Calle en gris
cmap[1] = np.array([256/256, 0/256, 0/256, 1])    # Semáforo en rojo
cmap[2] = np.array([256/256, 256/256, 0/256, 1])  # Semáforo en amarillo
cmap[3] = np.array([0/256, 256/256, 0/256, 1])    # Semáforo en verde
cmap[4] = np.array([0/256, 50/256, 100/256, 1])   # Carro azul
cmap[5] = np.array([230/256, 100/256, 20/256, 1]) # Carro naranja
cmap[6] = np.array([96/256, 170/256, 70/256, 1])  # Pasto verde

new_cmap = matplotlib.colors.ListedColormap(cmap)

fig, axs = plt.subplots(figsize=(10, 10))
axs.set_xticks([])
axs.set_yticks([])

patch = plt.imshow(all_grid.iloc[0][0], cmap=new_cmap)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])


anim = animation.FuncAnimation(fig, animate, frames=steps)

# Animación
Colores de celda:
- Gris: RobotLimpiador.
- Blanco: Baldosa limpia.
- Negro: Baldosa sucia. 

In [7]:
anim